In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tg.datasets import DatasetFactoryLookupCallback
from tg.model_interactor import ModelInteractor
from tg.splitters import AnchoredSplitter

In [ ]:
# "AIR_PASSENGERS"
# "PERFECT_SINE30"
# "NOISY_SINE30"
# "HOMICIDES"
# "RANDOM_WALK"

In [ ]:
dataset_name = "RANDOM_WALK"
data_factory = DatasetFactoryLookupCallback(dataset_name=dataset_name)

In [ ]:
# 'NAIVE'
# 'ARIMA'
# 'SARIMA'
# 'RNN'
# 'SVR'
# 'ELM'
# 'STL'
# 'ES'
# 'LSTM'
# 'ARIMA_RNN'
# 'SARIMA_SVR'
# 'STL_ELM'
# 'ES_LSTM'

In [ ]:
model_name = 'ES_LSTM'
params = {'trend': "mul", 'damped_trend': False, 'seasonal': None}


In [ ]:
y, X = data_factory(model_name=model_name)

mi = ModelInteractor(model_name=model_name)
mi.load(y=y,
        X=X,
        dataset_name=data_factory.dataset_name,
        timesteps=data_factory.dataset().period,
        train_size=data_factory.dataset().train_size,
        stack_size=data_factory.dataset().stack_size)

trains, test = mi.split_trains_test(
    y=mi.y,
    splitter_class=AnchoredSplitter,
    splitter_args={'min_train_points': data_factory.dataset().train_size},
    X=mi.X)

preds = mi.fit_predict(trains=trains, test=test, parameters=params)
metrics = mi.evaluate(preds=preds, test=test)

# mi.execute_mlflow(
#     splitter_class=AnchoredSplitter,
#     splitter_args={'min_train_points': data_factory.dataset.train_size},
#     parameters=mi.get_best_params())

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(y, label='y')
ax.plot(preds, label='preds')
ax.legend()

In [ ]:
mi.tune_hyperparameters(splitter_class=AnchoredSplitter,
                        splitter_args={
                            'min_train_points':
                            data_factory.dataset().tuning_train_size
                        },
                        n_trials=1)


In [ ]:
# mi.get_best_params()